# 8.2. Modelling with AutoGluon



In [1]:
import pandas as pd

In [10]:
df_train = pd.read_csv('./dataset/train_ts_feature_added.csv')
df_train['datetime_iso'] = pd.to_datetime(df_train['datetime_iso'])
df_train['datetime_iso'] = df_train['datetime_iso'].dt.tz_localize(None)
df_train['item_id'] = 'T1'

df_test = pd.read_csv('./dataset/test_ts_feature_added.csv')
df_test['datetime_iso'] = pd.to_datetime(df_test['datetime_iso'])
df_test['datetime_iso'] = df_test['datetime_iso'].dt.tz_localize(None)
df_test_date = df_test['datetime_iso']
df_test['item_id'] = 'T1'

In [11]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341880 entries, 0 to 341879
Data columns (total 19 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   datetime_iso     341880 non-null  datetime64[ns]
 1   temp             341880 non-null  float64       
 2   d_point          341880 non-null  float64       
 3   feels            341880 non-null  float64       
 4   min_temp         341880 non-null  float64       
 5   max_temp         341880 non-null  float64       
 6   prssr            341880 non-null  float64       
 7   hum              341880 non-null  float64       
 8   wind_spd         341880 non-null  float64       
 9   wind_deg         341880 non-null  float64       
 10  rain_1h          341880 non-null  float64       
 11  clouds           341880 non-null  float64       
 12  day_of_the_week  341880 non-null  int64         
 13  month            341880 non-null  int64         
 14  season           341

## Modeling

In [12]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [13]:
train_data = TimeSeriesDataFrame.from_data_frame(
    df_train,
    id_column="item_id",
    timestamp_column="datetime_iso"
)
train_data.head()

temp  d_point  feels  min_temp  max_temp  \
item_id timestamp                                                        
T1      1979-01-01 00:00:00  24.75    23.89  25.76     24.28     25.22   
        1979-01-01 01:00:00  24.58    23.73  25.57     23.99     25.26   
        1979-01-01 02:00:00  26.60    24.06  26.60     26.10     27.39   
        1979-01-01 03:00:00  27.31    24.37  30.90     26.59     28.36   
        1979-01-01 04:00:00  27.41    25.05  31.54     26.58     28.31   

                              prssr   hum  wind_spd  wind_deg  rain_1h  \
item_id timestamp                                                        
T1      1979-01-01 00:00:00  1012.0  95.0      0.82     320.0     0.00   
        1979-01-01 01:00:00  1012.0  95.0      0.96     338.0     0.00   
        1979-01-01 02:00:00  1012.0  86.0      1.22     339.0     0.00   
        1979-01-01 03:00:00  1012.0  84.0      1.08     342.0     0.13   
        1979-01-01 04:00:00  1011.0  87.0      0.86     336.0     0.34   

                             clouds  day_of_the_week  month  season  \
item_id timestamp                                                     
T1      1979-01-01 00:00:00   100.0                0      1       1   
        1979-01-01 01:00:00   100.0                0      1       1   
        1979-01-01 02:00:00    99.0                0      1       1   
        1979-01-01 03:00:00    94.0                0      1       1   
        1979-01-01 04:00:00   100.0                0      1       1   

                             yearly_mean  monthly_mean  weekly_mean  
item_id timestamp                                                    
T1      1979-01-01 00:00:00     0.492286      0.442561     0.485107  
        1979-01-01 01:00:00     0.492286      0.442561     0.485107  
        1979-01-01 02:00:00     0.492286      0.442561     0.485107  
        1979-01-01 03:00:00     0.492286      0.442561     0.485107  
        1979-01-01 04:00:00     0.492286      0.442561     0.485107

In [14]:
df_train.columns

Index(['datetime_iso', 'temp', 'd_point', 'feels', 'min_temp', 'max_temp',
       'prssr', 'hum', 'wind_spd', 'wind_deg', 'rain_1h', 'clouds',
       'day_of_the_week', 'month', 'season', 'yearly_mean', 'monthly_mean',
       'weekly_mean', 'item_id'],
      dtype='object')

In [16]:
df_test.describe()

,temp,d_point,feels,min_temp,max_temp,prssr,hum,wind_spd,wind_deg,rain_1h,clouds,day_of_the_week,month,season,yearly_mean,monthly_mean,weekly_mean
count,49368.000000,49368.000000,49368.000000,49368.000000,49368.000000,49368.000000,49368.000000,49368.000000,49368.000000,0.0,49368.000000,49368.000000,49368.000000,49368.000000,49368.000000,49368.000000,49368.000000
mean,27.297018,24.953872,30.442985,26.394385,28.362958,1010.069559,88.125020,1.315805,201.772201,NaN,92.503302,2.998542,6.279047,0.486631,0.371508,0.371532,0.371703
std,2.241756,0.848810,4.637139,2.099123,2.198794,1.754905,9.195383,0.606944,91.049427,NaN,11.752961,1.999412,3.402672,0.499826,0.108434,0.095352,0.100708
min,22.450000,22.560000,23.350000,20.040000,23.220000,1006.000000,63.000000,0.000000,0.000000,NaN,50.000000,0.000000,1.000000,0.000000,0.148857,0.211807,0.199193
25%,25.510000,24.370000,26.270000,24.760000,26.600000,1009.000000,82.000000,0.890000,156.000000,NaN,92.000000,1.000000,3.000000,0.000000,0.272553,0.292943,0.277556
50%,26.710000,24.920000,29.890000,25.950000,28.040000,1010.000000,91.000000,1.270000,206.000000,NaN,97.000000,3.000000,6.000000,0.000000,0.386709,0.381835,0.419342
75%,29.000000,25.510000,34.770000,28.020000,30.020000,1011.000000,95.000000,1.680000,270.000000,NaN,100.000000,5.000000,9.000000,1.000000,0.456165,0.453459,0.455577
max,34.210000,27.310000,41.200000,32.900000,35.140000,1014.000000,100.000000,3.060000,360.000000,NaN,100.000000,6.000000,12.000000,1.000000,0.622543,0.490919,0.552405


In [18]:
predictor = TimeSeriesPredictor(
    prediction_length=49368,
    path="autogluon-m4-hourly",
    target="rain_1h",
    known_covariates_names=['temp', 'd_point', 'feels', 'min_temp', 'max_temp',
       'prssr', 'hum', 'wind_spd', 'wind_deg', 'clouds',
       'day_of_the_week', 'month', 'season', 'yearly_mean', 'monthly_mean',
       'weekly_mean'],
    eval_metric="RMSE",
)

predictor.fit(
    train_data,
    presets="medium_quality",
    time_limit=600,
)

================ TimeSeriesPredictor ================
TimeSeriesPredictor.fit() called
Setting presets to: medium_quality
Fitting with arguments:
{'enable_ensemble': True,
 'evaluation_metric': 'RMSE',
 'excluded_model_types': None,
 'hyperparameter_tune_kwargs': None,
 'hyperparameters': 'medium_quality',
 'num_val_windows': 1,
 'prediction_length': 49368,
 'random_seed': None,
 'target': 'rain_1h',
 'time_limit': 600,
 'verbosity': 2}
Provided training data set with 341880 rows, 1 items (item = single time series). Average time series length is 341880.0. Data frequency is 'H'.
AutoGluon will save models to autogluon-m4-hourly/
AutoGluon will gauge predictive performance using evaluation metric: 'RMSE'
	This metric's sign has been flipped to adhere to being 'higher is better'. The reported score can be multiplied by -1 to get the metric value.

Provided dataset contains following columns:
	target:           'rain_1h'
	known covariates: ['temp', 'd_point', 'feels', 'min_temp', 'max_tem

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:12                                                                                   │
│                                                                                                  │
│    9 │   eval_metric="RMSE",                                                                     │
│   10 )                                                                                           │
│   11                                                                                             │
│ ❱ 12 predictor.fit(                                                                              │
│   13 │   train_data,                                                                             │
│   14 │   presets="medium_quality",                                                               │
│   15 │   time_limit=600,                                                                         │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/autogluon/core/utils/decorators.py:31 in _call           │
│                                                                                                  │
│   28 │   │   @functools.wraps(f)                                                                 │
│   29 │   │   def _call(*args, **kwargs):                                                         │
│   30 │   │   │   gargs, gkwargs = g(*other_args, *args, **kwargs)                                │
│ ❱ 31 │   │   │   return f(*gargs, **gkwargs)                                                     │
│   32 │   │                                                                                       │
│   33 │   │   return _call                                                                        │
│   34                                                                                             │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/autogluon/timeseries/predictor.py:531 in fit             │
│                                                                                                  │
│   528 │   │   │   set_random_seed(random_seed)                                                   │
│   529 │   │                                                                                      │
│   530 │   │   time_left = None if time_limit is None else time_limit - (time.time() - time_sta   │
│ ❱ 531 │   │   self._learner.fit(                                                                 │
│   532 │   │   │   train_data=train_data,                                                         │
│   533 │   │   │   val_data=tuning_data,                                                          │
│   534 │   │   │   hyperparameters=hyperparameters,                                               │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/autogluon/timeseries/learner.py:68 in fit                │
│                                                                                                  │
│    65 │   │   hyperparameter_tune_kwargs: Optional[Union[str, dict]] = None,                     │
│    66 │   │   **kwargs,                                                                          │
│    67 │   ) -> None:                                                                             │
│ ❱  68 │   │   return self._fit(                                                                  │
│    69 │   │   │   train_data=train_data,                                                         │
│    70 │   │   │   val_data=val_data,                                                             │
│    71 │   │   │   hyperparameters=hyperparameters,         